# Analyzing ICESat-2 snow depths over Alaska

This notebook is designed to take the frameworks from the SlideRule and icepyx querying notebooks to examine ICESat-2 snow depths over SnowEx field sites in Alaska. 

Snow-off/-on lidar DEMs from the UAF lidar are required for the running of this script. Note that the airborne lidar data is still preliminary, so it is currently not available publicly. Until the data is posted to NSIDC, it is recommended for any users to reach out to the original provider of the data (Chris Larsen, cflarsen@uaf.edu), and be aware that there may be a few oddities in the DEMs.

In [ ]:
# General packages
import geopandas as gpd
import holoviews as hv
from holoviews import opts, Cycle
import icepyx as ipx
from is2_cloud_access import atl03q, atl06q, atl08q
import lidar_processing as lp
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pyproj import Proj, transform, Transformer, CRS
import rioxarray as rio
from shapely import wkt
from shapely.geometry import Polygon, Point
import sys
import xarray as xr
hv.extension('bokeh')

# SlideRule-relevant packages
import ipywidgets as widgets
import logging
import concurrent.futures
import time
from datetime import datetime
from sliderule import icesat2
from sliderule import sliderule, ipysliderule, io
import geoviews as gv
import geoviews.feature as gf
from geoviews import dim, opts
import geoviews.tile_sources as gts
from bokeh.models import HoverTool
import hvplot.pandas
gv.extension('bokeh')

### User Input

Acceptable field site IDs over Alaska are:
* 'cffl': Creamer's Field/Farmer's Loop
* 'cpcrw': Caribou/Poker Creek Experimental Watershed
* 'bcef': Bonanza Creek Experimental Forest
* 'trs': Toolik Research Station
* 'acp': Arctic Coastal Plain (Deadhorse)

**NOTE**: Functionality for Caribou/Poker Creek and Toolik data is a work in progress.

In [ ]:
# Field site ID
field_id = 'acp'

# Snow-on (True) or snow-off (False) analysis
snow_on = True

# Base data path
path = '/home/jovyan/icesat2-snowex/'

# Desired RGT and date range for queries (needed to filter SlideRule and for icepyx query)
date_range = ['2022-03-01', '2022-04-30']
rgt = '1097'

# Save the data to a CSV?
saving = True

### Read UAF Lidar Data

Note that the snow-off lidar DEM is needed to estimate ICESat-2 snow depths, so it must be available even for snow-on analyses.

A correction factor is applied to the snow-off DEM to account for vertical datum differences. It is currently field-site dependent until a more robust calibration is derived.

Field site data will be added to this segment when it becomes available.

In [ ]:
if field_id == 'cffl':
    f_snow_off = '%s/lidar-dems/farmersloop_2022may28_dtm_3m.tif' %(path)
    f_snow_on = '%s/lidar-dems/farmersloop_2022mar11_snowdepth_3m.tif' %(path)
    
    # Vertical datum correction factor
    lidar_correction_factor = 9.75
    
elif field_id == 'bcef':
    f_snow_off = '%s/lidar-dems/bonanza_2022may28_dtm_3m.tif' %(path)
    f_snow_on = '%s/lidar-dems/bonanza_2022mar11_snowdepth_3m.tif' %(path)
    
    # Vertical datum correction factor
    lidar_correction_factor = 9.9
    
elif field_id == 'acp':
    f_snow_off = '%s/lidar-dems/coastalplain_2022aug31_dtm_3m.tif' %(path)
    f_snow_on = '%s/lidar-dems/coastalplain_2022mar12_snowdepth_3m.tif' %(path)
    
    # Vertical datum correction factor
    lidar_correction_factor = 9.9
    print('CAUTION: ICESat-2/UAF calibration has not been performed for this field site. Snow-off results may be inaccurate.')

In [ ]:
# Read lidar DEMs into xarray (rasterio) format
lidar_snow_off = rio.open_rasterio(f_snow_off)
lidar_snow_on = rio.open_rasterio(f_snow_on)

In [ ]:
lidar_snow_off

In [ ]:
lidar_snow_on

### Read ICESat-2 Data

Again, reading of the ICESat-2 data will be site specific. These cells will be reading in **ATL03, ATL06, and ATL08** data. ATL03 will be accessed and processed using SlideRule, whereas icepyx will provide ATL06 and ATL08. Both of these software packages will consolidate the ICESat-2 products under the hood, for cleanliness.

In [ ]:
# Query the ICESat-2 data products
atl03 = atl03q(field_id)
atl06 = atl06q(field_id, date_range, rgt)
atl08 = atl08q(field_id, date_range, rgt)

In [ ]:
atl03.head()

In [ ]:
atl06.head()

In [ ]:
atl08.head()

### Processing ICESat-2 data

Now that we have batches of ICESat-2 data for ATL03, ATL06, and ATL08, we will now do some filtering to only look at our RGT, time, and region of interest in all three products. We will also add easting/northing coordinates to each DataFrame.

In [ ]:
# Add easting/northing coordinates to ATL03 DataFrame
atl03['x'] = atl03.to_crs(epsg=32606).geometry.x
atl03['y'] = atl03.to_crs(epsg=32606).geometry.y

atl03.head()

In [ ]:
# Add easting/northing coordinates to ATL06/08 DataFrames
#inp = Proj('epsg:4326')
#outp = Proj('epsg:32606')

transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32606', always_xy=True)

atl06['x'], atl06['y'] = transformer.transform(atl06.lon, atl06.lat)
atl08['x'], atl08['y'] = transformer.transform(atl08.lon, atl08.lat)

print(atl06.head())
print(atl08.head())

In [ ]:
# Filter ATL03 data by RGT and date range
atl03 = atl03.loc[atl03['rgt'] == int(rgt)]
atl03 = atl03[atl03.index.to_series().between(date_range[0], date_range[1])]
atl03.head()

In [ ]:
# Include only the strong beams in the ATL03 DataFrame
strong_spots = ['1', '3', '5']
atl03['spot'] = atl03['spot'].apply(str)
atl03 = atl03[atl03['spot'].isin(strong_spots)]

# Change the ATL03 height column name to be consistent with other DataFrames
atl03 = atl03.rename(columns={'h_mean': 'height'})
atl03.head()

In [ ]:
## Remove filler/messy data from ATL06/08
# ATL06
upper = atl06.height.mean() + 3*atl06.height.std()
atl06 = atl06.loc[atl06.height<upper]

# ATL08
upper = atl08.height.mean() + 3*atl08.height.std()
atl08 = atl08.loc[atl08.height<upper]

In [ ]:
# Limit ATL06/08 to ATL03 spatial bounds
atl06 = atl06[(atl06.y.values>atl03.y.min()) & (atl06.y.values<atl03.y.max())]
atl08 = atl08[(atl08.y.values>atl03.y.min()) & (atl08.y.values<atl03.y.max())]

print(atl06.head())
print(atl08.head())

### Process UAF Lidar Data

Here, we are going to apply the lidar correction factor to the snow-off lidar data, then coregister the DEMs with each of the ICESat-2 data products.

In [ ]:
# Apply vertical datum correction
lidar_snow_off -= lidar_correction_factor

# Coregister with ICESat-2 using a bivariate spline
strong_ids = np.unique(atl06['gt'])
atl03_uaf = lp.coregister_is2(lidar_snow_off, lidar_snow_on, atl03, strong_ids)
atl06_uaf = lp.coregister_is2(lidar_snow_off, lidar_snow_on, atl06, strong_ids)
atl08_uaf = lp.coregister_is2(lidar_snow_off, lidar_snow_on, atl08, strong_ids)

# Calculate snow depth residuals
if snow_on:
    atl03_uaf['snow_depth_residual'] = atl03_uaf['residual'] - atl03_uaf['lidar_snow_depth']
    atl06_uaf['snow_depth_residual'] = atl06_uaf['residual'] - atl06_uaf['lidar_snow_depth']
    atl08_uaf['snow_depth_residual'] = atl08_uaf['residual'] - atl08_uaf['lidar_snow_depth']

In [ ]:
print(atl03_uaf.head())
print(atl06_uaf.head())
print(atl08_uaf.head())

## Statistical Filtering

The interpolation included some messy data or filler values, so this is a short section to only include data within the 10th and 90th percentiles.

In [ ]:
# ATL03
upper = atl03_uaf['residual'].quantile(0.9)
lower = atl03_uaf['residual'].quantile(0.1)
atl03_uaf_filtered = atl03_uaf[(atl03_uaf['residual']>=lower) &
                               (atl03_uaf['residual']<=upper) &
                               (atl03_uaf['lidar_snow_depth']>0)]

# ATL06
upper = atl06_uaf['residual'].quantile(0.9)
lower = atl06_uaf['residual'].quantile(0.1)
atl06_uaf_filtered = atl06_uaf[(atl06_uaf['residual']>=lower) &
                               (atl06_uaf['residual']<=upper) &
                               (atl06_uaf['lidar_snow_depth']>0)]

# ATL08
upper = atl08_uaf['residual'].quantile(0.9)
lower = atl08_uaf['residual'].quantile(0.1)
atl08_uaf_filtered = atl08_uaf[(atl08_uaf['residual']>=lower) &
                               (atl08_uaf['residual']<=upper) &
                               (atl08_uaf['lidar_snow_depth']>0)]

print(atl03_uaf_filtered.head())
print(atl06_uaf_filtered.head())
print(atl08_uaf_filtered.head())

## Saving snow depth data
The final aim for this notebook is to develop an ICESat-2 snow depth "product" over the Alaska field sites. This is still very much a work in progress, but the section still allows for saving of derived snow depth data.

In [ ]:
# Save derived snow depth data in .csv format
atl03_uaf_filtered.to_csv(f'atl03_snowdepth_rgt{rgt}_{field_id}_{time_str}.csv')
atl06_uaf_filtered.to_csv(f'atl06_snowdepth_rgt{rgt}_{field_id}_{time_str}.csv')
atl08_uaf_filtered.to_csv(f'atl08_snowdepth_rgt{rgt}_{field_id}_{time_str}.csv')

## Plotting

Now that we have processed and co-registered the two lidar data sets, let's make some plots!

In [ ]:
# Add latitude, longitude coordinates to DataFrames
transformer = Transformer.from_crs('EPSG:32606', 'EPSG:4326', always_xy=True)

atl03_uaf_filtered['lon'], atl03_uaf_filtered['lat'] = transformer.transform(atl03_uaf_filtered.x,
                                                                             atl03_uaf_filtered.y)
atl06_uaf_filtered['lon'], atl06_uaf_filtered['lat'] = transformer.transform(atl06_uaf_filtered.x,
                                                                             atl06_uaf_filtered.y)
atl08_uaf_filtered['lon'], atl08_uaf_filtered['lat'] = transformer.transform(atl08_uaf_filtered.x,
                                                                             atl08_uaf_filtered.y)

# Convert DataFrames to GeoDataFrames
# ATL03
geometry = [Point(xy) for xy in zip(atl03_uaf_filtered.lon, atl03_uaf_filtered.lat)]
atl03_uaf_gpd = gpd.GeoDataFrame(atl03_uaf_filtered, geometry=geometry)

# ATL06
geometry = [Point(xy) for xy in zip(atl06_uaf_filtered.lon, atl06_uaf_filtered.lat)]
atl06_uaf_gpd = gpd.GeoDataFrame(atl06_uaf_filtered, geometry=geometry)

# ATL08
geometry = [Point(xy) for xy in zip(atl08_uaf_filtered.lon, atl08_uaf_filtered.lat)]
atl08_uaf_gpd = gpd.GeoDataFrame(atl08_uaf_filtered, geometry=geometry)

In [ ]:
# Read in the SnowEx lidar boxes
lidar_boxes = gpd.read_file('/home/jovyan/icesat2-snowex/jsons-shps/snowex_lidar_swaths.shp')

# Geoviews map of snow depth over site
hover = HoverTool(tooltips=[('Latitude', '@Latitude'),
                             ('Longitude', '@Longitude'),
                             ('is2_snow_depth', '@residual')])
lidar_boxes_poly = gv.Polygons(lidar_boxes).opts(color='white',
                                                 alpha=0.5)
points_on_map = gv.Points(atl03_uaf_gpd,
                           kdmins=['Longitude', 'Latitude'],
                           vdmins=['residual']).opts(tools=[hover],
                                                     color_index='residual',
                                                     colorbar=True,
                                                     clabel='IS-2 snow depth [m]',
                                                     size=4.0)


world_map = gts.EsriImagery.opts(width=600, height=570) * gts.StamenLabels.options(level='annotation')

world_map * lidar_boxes_poly * points_on_map

In [ ]:
# Generate histogram bins for ICESat-2 snow depth data
freq, edges = np.histogram(atl03_uaf_filtered['residual'], 100)
freq06, edges06 = np.histogram(atl06_uaf_filtered['residual'], 100)
freq08, edges08 = np.histogram(atl08_uaf_filtered['residual'], 100)
frequaf, edgesuaf = np.histogram(atl03_uaf_filtered['lidar_snow_depth'][abs(atl03_uaf_filtered['lidar_snow_depth'])<3], 100)

# Generate scatter plot for UAF vs. ICESat-2 snow depth
scatter = atl03_uaf_filtered.hvplot(kind='scatter', x='lidar_snow_depth', y='residual', label='ATL03', alpha=0.5)
scatter06 = atl06_uaf_filtered.hvplot(kind='scatter', x='lidar_snow_depth', y='residual', label='ATL06', alpha=0.5)
scatter08 = atl08_uaf_filtered.hvplot(kind='scatter', x='lidar_snow_depth', y='residual', label='ATL08', alpha=0.5)
scatters = (scatter*scatter06*scatter08).opts(xlabel='UAF snow depth [m]', ylabel='ICESat-2 snow depth [m]')

# Generate PDF curves for snow depth
curves = (hv.Distribution((edges, freq), label='ATL03').opts(bandwidth=0.6)) * \
         (hv.Distribution((edges06, freq06), label='ATL06').opts(bandwidth=0.6)) * \
         (hv.Distribution((edges08, freq08), label='ATL08').opts(bandwidth=0.6)) * \
         (hv.Distribution((edgesuaf, frequaf), label='UAF').opts(bandwidth=0.6))
curves.opts(xlabel='Snow depth [m]', ylabel='PDF')

scatters + curves